<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
#Import packages
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.support.wait import WebDriverWait

from datetime import datetime
import pandas as pd
import numpy as np
import random
import time
import os
import re

In [5]:
options = webdriver.ChromeOptions()
capabilities = options.to_capabilities()
capabilities = {
 'browser': 'chrome',
 'browser_version': 'latest',
 'os': 'Windows',
 'os_version': '10',
 'build': 'Python Sample Build',
 'name': 'Pop-ups testing'
}
#capabilities["chromeOptions"]["excludeSwitches"] = ["disable-popup-blocking"]

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.maximize_window()

url = "https://www.kawalingpinoy.com/recipe-index/"

driver.get(url)
driver.implicitly_wait(8)

page = 1

while True:
    try:
        print(f'Harvesting page: {page}....', end="")
        
        urls = []
        
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        for i in range(1, total_height, 15):
            driver.execute_script("window.scrollTo(0, {});".format(i))
            
        recipe = driver.find_elements(By.XPATH, '//header[@class="entry-header"]//a[@rel = "bookmark"]')

        for _recipe in recipe:
            urls.append({"url":_recipe.get_attribute('href')})
            
        url_df = pd.DataFrame(urls)
        url_df.to_csv("recipe_data/kawaling_pinoy_" + str(page) + ".csv", index=False)
        print("Saved!")
        
        start_url = driver.current_url
        time.sleep(0.5)

        next_page = driver.find_element(By.XPATH, "//li[@class='pagination-next']")
        next_page.click()
        time.sleep(1)
        
        
        time.sleep(1)
        
        next_url = driver.current_url
        if start_url == next_url:
            print(f"Scraper stopped moving, scaping ended at page {page}")
            break

        page += 1
        
    except:
        print("Scraping stopped moving")
        break
        


C:\Users\Emong\AppData\Local\Temp\ipykernel_17996\3753864439.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


Harvesting page: 1....Saved!
Harvesting page: 2....Saved!
Harvesting page: 3....Saved!
Harvesting page: 4....Saved!
Harvesting page: 5....Saved!
Harvesting page: 6....Saved!
Harvesting page: 7....Saved!
Harvesting page: 8....Saved!
Harvesting page: 9....Saved!
Harvesting page: 10....Saved!
Harvesting page: 11....Saved!
Harvesting page: 12....Saved!
Harvesting page: 13....Saved!
Harvesting page: 14....Saved!
Harvesting page: 15....Saved!
Harvesting page: 16....Saved!
Harvesting page: 17....Saved!
Harvesting page: 18....Saved!
Harvesting page: 19....Saved!
Harvesting page: 20....Saved!
Harvesting page: 21....Saved!
Harvesting page: 22....Saved!
Harvesting page: 23....Saved!
Harvesting page: 24....Saved!
Harvesting page: 25....Saved!
Harvesting page: 26....Saved!
Harvesting page: 27....Saved!
Harvesting page: 28....Saved!
Harvesting page: 29....Saved!
Harvesting page: 30....Saved!
Harvesting page: 31....Saved!
Harvesting page: 32....Saved!
Harvesting page: 33....Saved!
Harvesting page: 34

In [14]:
url = pd.DataFrame({"url":[]})

for i in range(1,35):
    df = pd.read_csv("recipe_data/kawaling_pinoy_"+str(i)+".csv")
    url = pd.concat([url, df], axis=0)
    
data_url = url.reset_index(drop=True)


In [17]:
driver = webdriver.Chrome(ChromeDriverManager().install())
recipes = []

for food in data_url["url"]:
    driver.get(food)
    time.sleep(2)
    try:
        meal = driver.find_element(By.XPATH,"//div[@class='wprm-recipe-custom-card-by-once-coupled']//h2[@class='wprm-recipe-name wprm-block-text-normal']").text
    except:
        meal = 'none'
    #description = driver.find_element(By.XPATH,"//div[@class='wprm-recipe-custom-card-by-once-coupled']//div[@class='wprm-recipe-summary wprm-block-text-normal']").text
    #prep_and_cook_time = driver.find_elements(By.XPATH,"//div[@class='wprm-recipe-custom-card-by-once-coupled']//span[@class='wprm-recipe-time wprm-block-text-normal']")
    try:
        course = driver.find_element(By.XPATH,"//div[@class='wprm-recipe-custom-card-by-once-coupled']//span[@class='wprm-recipe-course wprm-block-text-normal']").text
    except:
        course = 'none'
    #review = driver.find_element(By.XPATH,"//div[@class='wprm-recipe-custom-card-by-once-coupled']//span[@class='wprm-recipe-rating-average']").text
    #number_of_review = driver.find_element(By.XPATH,"//div[@class='wprm-recipe-custom-card-by-once-coupled']//span[@class='wprm-recipe-rating-count']").text
    try:
        instruction = driver.find_element(By.XPATH,"//div[@class='wprm-recipe-custom-card-by-once-coupled']//ul[@class='wprm-recipe-instructions']").text
    except:
        instruction = 'none'
    try:
        ingredient_list = driver.find_elements(By.XPATH,"//div[@class='wprm-recipe-custom-card-by-once-coupled']//li[@class='wprm-recipe-ingredient']")
        ingredients = []
        for i in range(0, len(ingredient_list)):
            ingredients.append(ingredient_list[i].text)
        ingredients = ', '.join(ingredients)
    except:
        ingredients = 'none'
    
    recipes.append({
        "url": food,
        "food": meal,
        "course": course,
        #"description": description,
        #"prep time": prep_and_cook_time[0].text,
        #"cook time": prep_and_cook_time[1].text,
        #"number of serving": number_of_serving, 
        #"calories": calories,
        "ingredients": ingredients,
        "instructions": instruction
        })
    time.sleep(2)

kawaling_pinoy_df = pd.DataFrame(recipes)
kawaling_pinoy_df.to_csv("recipe_data/kawaling_pinoy.csv", index=False)

C:\Users\Emong\AppData\Local\Temp\ipykernel_3104\53874205.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [16]:
recipes

[{'url': 'https://www.kawalingpinoy.com/salted-egg-shrimp/',
  'food': 'Salted Eggs Shrimp',
  'course': 'Main Entree',
  'ingredients': '1 pound large shrimp, 2 tablespoons butter, 1 tablespoon oil, 2 shallots, peeled and finely chopped, 2 cloves garlic, peeled and minced, 2 Thai chili peppers, stemmed and minced, 1 cup evaporated milk, 6 salted egg yolks, mashed, 1/2 teaspoon sugar, salt to taste',
  'instructions': 'Trim tendrils of shrimp and rinse under cold water. In a colander, drain very well.\nIn a wide pan over medium heat, heat butter and oil.\nWhen butter begins to melt, Add shallots, garlic, and chili peppers. Cook, stirring regularly, for about 1 minute or until softened.\nReduce heat, add evaporated milk, and bring to a simmer.\nAdd salted egg yolks and continue to cook, mashing yolks with back of spoon, for about 1 minute or until slightly thickened.\nAdd sugar and season with salt to taste.\nAdd shrimp and cook, stirring occasionally, until color changes to pink and th

In [10]:
len(recipes)

594

In [37]:
final_data.isna().sum()

url             0
food            0
prep time       0
cook time       0
ingredients     0
instructions    0
dtype: int64